In [1]:
import pandas as pd
import numpy as np
import random as rd 
import pickle as pk

from Judge.Score import *   # 评价指标

In [2]:
df_ft_origin = pd.read_csv('./Datasets/train_feature.csv')
df_lb_origin = pd.read_csv('./Datasets/train_label.csv') 

# df_origin = pd.concat([df_ft_origin, df_lb_origin],axis=1)


## 数据预处理
首先进行数据的预处理。

In [3]:
from Preprocessing import Tools as M_TOOL
from Preprocessing import PCA as M_PCA

df_ft = M_TOOL.random_FillMissingData(df_ft_origin)
df_lb = M_TOOL.random_FillMissingData(df_lb_origin)

df_ft = M_TOOL.Normalization(df_ft).values

PCA_model = M_PCA.PCA(df_ft)
df_ft = pd.DataFrame(PCA_model.reduce_dimension())
# print(df.head())
df = pd.concat([df_ft, df_lb], axis=1)
df.drop_duplicates(inplace=True)
print(df.shape)


X_train, y_train, X_test, Y_test = M_TOOL.random_Split_data(df)


(10000, 115)


## 模型训练与预测

接下来，我们将分别使用线性回归模型、决策树模型、神经网络模型、支持向量机以及 XGBoost 等分类模型对数据集进行训练，并在验证集上进行预测。

In [4]:
from Methods import LinearReg as M_LR
# 线性回归模型

model_liner = M_LR.LinearRegression(X_train, y_train)
model_liner.fit()
# model_liner.save()
# model_liner = pk.load(open("./model/Model_LinearReg_2022_12_24_21_07_58.dat", "rb"))    # 使用已保存的模型

pre = model_liner.predict(X_test)



acc = Accuracy(pre, Y_test)
print("线性回归模型分类准确度为 {}".format(acc))


# 与 Sklearn 比较
from sklearn.linear_model import LinearRegression as LR  
model_liner_skl = LR().fit(X_train, y_train)
pre_skl = model_liner_skl.predict(X_test)

acc = Accuracy(pre_skl, Y_test)
print("SKLearn 分类准确度为 {}".format(acc))


线性回归模型分类准确度为 0.25560000000000005
SKLearn 分类准确度为 0.24160000000000004


In [5]:
# from Methods import RegTree as M_RT
# # 回归树模型

# m, n = y_train.shape 
# y_t = np.zeros((m, n))
# model_regtree = M_RT.RegressionTree(X_train, y_train, y_t)
# default_parameters = {
#     "lamda": 2,             # Hyperparameters
#     "gamma": 1e-6,          # Hyperparameters
#     "gain_delta": 0,        # The minimum gain
#     "max_depth": 3,
#     "max_leaves": 100,
#     "max_nodes": 1000,
#     "min_samples": 10,      # Minimum number of samples on a leaf
#     "min_feature_dif": 5,   # The minimum number of different value for current feature
#     "if_silent": 0
# }
# model_regtree.fit(default_parameters)
# pre = model_regtree.predict(X_test)

# acc = Accuracy(pre, Y_test)
# print("回归树模型分类准确度为 {}".format(acc))


In [6]:
# from Methods import SVM as M_SVM
# # 支持向量机模型

# model_SVM = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM.fit(max_times=500, ifsilent=True)
# pre = model_SVM.predict(X_test)

# acc = Accuracy(pre, Y_test)
# print("SVM 模型分类准确度为 {}".format(acc))


# # 与 SKLearn 比较
# from sklearn import svm 
# model = svm.SVC(kernel='linear')
# model.fit(X_train, y_train.flatten())
# pre_skl = model.predict(X_test).reshape(-1, 1)

# acc = Accuracy(pre_skl, Y_test)
# print("SKLearn 分类准确度为 {}".format(acc))

In [7]:
# from Methods import XGBoost as M_XGB
# # XGBoost 模型

# model_XGB = M_XGB.XGBoost(X_train, y_train)
# tree_parameters = {
#     "lamda": 1,         
#     "gamma": 0,          
#     "gain_delta": 0,     
#     "max_depth": 2,
#     "max_leaves": 100,
#     "max_nodes": 1000,
#     "min_samples": 3,      
#     "min_feature_dif": 3,  
#     "if_silent": 0,
# }
# model_XGB.fit(T=35, tree_parameters=tree_parameters, min_train_err=1e-4)
# pre = model_XGB.predict(X_test)

# acc = Accuracy(pre, Y_test)
# print("线性回归模型分类准确度为 {}".format(acc))
